# Visualising reference genome
__1.__ 195,471,971 bp  
__2.__ Use 'grep' to print out the fasta header lines

`grep '>' GRCm38.68.dna.toplevel.fa` 

This will show you 
* 'standard' chromosomes, 1, 2 ...X, Y, MT
and 
* unassembled scaffolds: JH584295.1, ... JH584299.1

# Aligning paired FASTQ files with BWA

__1.__ `wc -l fastq/*
   785640 fastq/md5638a_7_87000000_R1.fastq
   785640 fastq/md5638a_7_87000000_R2.fastq
  157128 total`
  This says there are 157128 / 4 = 39,282 input reads

__2.__ `grep -c -v '@' md5638.sam` yields the count of 393833 lines. 

__3.__ Most likely there are multiple alignments per read. We can prove this by counting up how many duplicated read-names there are:

`grep -v '@' md5638.sam | cut -f 1 | less | sort | uniq -c | sort -k 1,1nr > aligned_read_counts_by_read_name.txt`

Now browse the `aligned_read_counts_by_read_name.txt` file with less:

`less aligned_read_counts_by_read_name.txt`

and notice that there are read names with 5 alignments, 4 alignments etc. 

      5 HX8_24050:2:1124:3244:50392
      5 HX8_24050:3:2218:23815:30896
      5 HX8_24050:4:1219:23754:3612
      5 HX8_24050:4:2115:5386:23477
      5 HX8_24050:4:2224:4279:54787
      4 HX8_24050:1:2118:5325:21508
      4 HX8_24050:1:2124:5812:4561
      ...
      2 HX8_24050:5:2224:9587:45628
      2 HX8_24050:5:2224:9709:33850
      2 HX8_24050:5:2224:9820:15303
      2 HX8_24050:5:2224:9881:6970
      2 HX8_24050:5:2224:9932:1080
      2 HX8_24050:5:2224:9973:16270


Read-names describe both members of a read-pair - We are expecting 2 alignments per read-name - so these high counts result from multiple alignments for the same read.



# Converting a SAM file to a BAM file

__1.__ Simply typing `samtools view` will bring up the help options:

    samtools view
    Usage: samtools view [options] <in.bam>|<in.sam>|<in.cram> [region ...]
    Options:
      -b       output BAM
      ...

Reading through the options suggests we should use the -O bam and -o <file name> options.

__2.__ The `ls` commands with arguments `-lh` will show the number of bytes used by each file listed, in a human-readable form:

    ls -lh
    total 188M
    -rw-r--r-- 1 vvi hgi 6.9M Aug 26 11:05 aligned_read_counts_by_read_name.txt
    -rw-r--r-- 1 vvi hgi  26M Aug 26 11:34 md5638.bam
    -rw-r--r-- 1 vvi hgi 157M Aug 25 16:29 md5638.sam

In this case you can see that the bam file takes up 26Mb, whereas the sam file takes up 157Mb, so converting to bam is a 6x space saving!


__3__. No reads where lost by converting SAM to BAM:

    samtools view -c md5638.bam 
    393833

    samtools view -c md5638.sam
    393833

# Sorting and indexing the BAM file

__1.__ use the samtools index command

In [ ]:
samtools index md5638.sorted.bam

__2.__ We can quickly see that there are alignments to many chromsomes, not just chr7 (which is where the mouse Tyr gene is located).

In [ ]:
samtools view md5638.sorted.bam | head | less -S
samtools view md5638.sorted.bam | tail | less -S

    HX8_24050:5:1123:17036:64333    163     1       4559741 0       65M86S 
    HX8_24050:4:1120:23571:25341    99      1       4559735 0       66M85S 
    ...
    ...
    HX8_24050:3:1215:11749:58585    163     Y       60702321        0       151M
    HX8_24050:3:1215:11749:58585    83      Y       60702549        0       151M

This is because both the intronic sections and the exonic sections of the gene will contain matches - perfect or otherwise - to other regions of the genome. The aligner will report all of those matches.

We can quickly see that the vast majority of the alignments are on chr7:

In [ ]:
 samtools view md5638.sorted.bam | cut -f 3 | uniq -c

In [ ]:
 
    835 1
    467 10
    379 11
    487 12
    480 13
    479 14
    426 15
    299 16
    189 17
    284 18
     98 19
    430 2
    520 3
    492 4
    533 5
    443 6
 385335 7 <<<<< SEE HERE, most of the aligns are on chr7
    322 8
    347 9
    775 X
    198 Y
      2 GL456212.1
      6 JH584303.1
      2 GL456219.1
      2 GL456354.1
      2 JH584297.1
      1 GL456221.1

# Marking PCR duplicates

__1.__ The metrics file shows that 33490 read-pairs out of 195184 read-pairs were marked as PCR duplicates, which is a duplicate rate of ~17%

    ## METRICS CLASS        picard.sam.DuplicationMetrics
    LIBRARY UNPAIRED_READS_EXAMINED READ_PAIRS_EXAMINED     SECONDARY_OR_SUPPLEMENTARY_RDS  UNMAPPED_READS  UNPAIRED_READ_DUPLICATES        READ_PAIR_DUPLICATES
    READ_PAIR_OPTICAL_DUPLICATES    PERCENT_DUPLICATION     ESTIMATED_LIBRARY_SIZE
    Unknown Library 1226    195184  1013    1226    358     33490   12796   0.171959        741723

# Generating QC stats

__1.__ 392820 - this is same as the number from the input fastq files = 2 x (785640/4):

    wc -l ../fastq/md5638a_7_87000000_R1.fastq 
    785640 ../fastq/md5638a_7_87000000_R1.fastq

It is _not_ the same as the number of alignments in the input bam file:

    samtools view -c md5638.markdup.bam 
    393833
    
__2.__ 391594 / 392820 ~ 99.7% of the reads were mapped.

__3.__ 389366

__4.__ 37

__5.__ Insert size mean: 418.4 SD: 113.0

__6.__ 2.7%

__7.__ First fragment


# BAM visualisation

__1.__ Variable from high-50's to high-20's.

__2.__ There's a 1bp insertion of a "T" in 9 reads, a 28bp deletion in 3 reads and a big (300bp) deletion with evidence from 6 read-pairs with trucated / soft-clipped insert-size reads & a stack more purely soft-clipped read-pairs.

__3.__ As we have said, these mutations are the result of CRISPR-cas9 acting to cut the genome roughly in the middle of the area we are looking at. That cutting resulted in Non-Homologous-End-Join-based (NHEJ) damage around base 87,483,960: that resulted in a subclonal 1bp insertion and a 28bp deletion. Microhomology–induced-end-joining resulted in the 338bp deletion.

Can you see the “TTT” motif on the 5’ end of the deletion, and just inside the 3’ end of the deletion? You are watching the zygote DNA-repair machinery panicking and grabbing at straws).

Why are these alleles subclonal? Because the action of the CRISPR-Cas9 occurred both at the single-cell and the two-cell stage.